#### Connect with database and ask some basic questions

In [29]:

#from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

from langchain.utilities import SQLDatabase


In [ ]:
db_user = "root"
db_password = "*******"
db_host = "localhost"
db_name = "retail_sales_db"

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)


print(db.table_info)


CREATE TABLE sales_tb (
	`TransactionID` INTEGER, 
	`Date` DATE, 
	`CustomerID` VARCHAR(10), 
	`Gender` VARCHAR(10), 
	`Age` INTEGER, 
	`ProductCategory` VARCHAR(50), 
	`Quantity` INTEGER, 
	`PriceperUnit` DECIMAL(10, 2), 
	`TotalAmount` DECIMAL(10, 2)
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from sales_tb table:
TransactionID	Date	CustomerID	Gender	Age	ProductCategory	Quantity	PriceperUnit	TotalAmount
1	2023-11-24	CUST001	Male	34	Beauty	3	50.00	150.00
2	2023-02-27	CUST002	Female	26	Clothing	2	500.00	1000.00
3	2023-01-13	CUST003	Male	50	Electronics	1	30.00	30.00
*/


In [ ]:
from langchain_groq import ChatGroq
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase


# Use a supported model
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", api_key="gsk_6z*************3FYl5kO5sj6h18yN71qUyhVmhvC")

# Create the chain
chain = create_sql_query_chain(llm, db)

# Ask a question
response = chain.invoke({"question": "How many customers are there?"})
print(response)


Here is the answer:

Question: How many customers are there?
SQLQuery: SELECT COUNT(DISTINCT `CustomerID`) AS `Number of Customers` FROM `sales_tb` LIMIT 1;


In [32]:
cleaned_query = response.strip('```sql\n').strip('\n```')
print(cleaned_query)

Here is the answer:

Question: How many customers are there?
SQLQuery: SELECT COUNT(DISTINCT `CustomerID`) AS `Number of Customers` FROM `sales_tb` LIMIT 1;


In [33]:
from sqlalchemy import create_engine, inspect

engine = create_engine(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
inspector = inspect(engine)

print(inspector.get_table_names())


['sales_tb']


In [34]:
import re

# Sample response from the LLM
# response = """Here is the answer:

# Question: How many customers are there?
# SQLQuery: SELECT COUNT(*) FROM customers
# """

# Extract only the SQL query using a regex
match = re.search(r"SELECT .*", response, re.IGNORECASE | re.DOTALL)
cleaned_query = match.group(0) if match else None

if cleaned_query:
    print("Cleaned SQL Query:", cleaned_query)
    result = db.run(cleaned_query)
    print(result)
else:
    print("No valid SQL query found in the response.")


Cleaned SQL Query: SELECT COUNT(DISTINCT `CustomerID`) AS `Number of Customers` FROM `sales_tb` LIMIT 1;
[(29,)]


In [37]:
# from sqlalchemy.exc import ProgrammingError

In [39]:
from sqlalchemy.exc import ProgrammingError
import re

def execute_query(question):
    try:
        # Generate SQL query from question
        response = chain.invoke({"question": question})
        print(response)
        print("###################################################")
        
        # Extract the actual SQL query using regex
        match = re.search(r"SELECT .*", response, re.IGNORECASE | re.DOTALL)
        cleaned_query = match.group(0) if match else None
        print(cleaned_query)
        print("###################################################")        
        
        if cleaned_query:
            # Execute the cleaned query
            result = db.run(cleaned_query)
            print("###################################################")        
            # Display the result
            print(result)
        else:
            print("No valid SQL query found in the response.")

    except ProgrammingError as e:
        print(f"An error occurred: {e}")


In [40]:
q1 = "How many unique customers are there for each product category"
execute_query(q1)

Here is the answer:

Question: How many unique customers are there for each product category
SQLQuery: SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `UniqueCustomers` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
SELECT `ProductCategory`, COUNT(DISTINCT `CustomerID`) AS `UniqueCustomers` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
###################################################
[('Beauty', 8), ('Clothing', 13), ('Electronics', 8)]


In [41]:
q2 = "Calculate total sales amount per product category:"
execute_query(q2)

Here is the answer:

Question: Calculate total sales amount per product category:
SQLQuery: SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSalesAmount` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSalesAmount` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
###################################################
[('Beauty', Decimal('1455.00')), ('Clothing', Decimal('5040.00')), ('Electronics', Decimal('5310.00'))]


In [42]:
q3 = "calculates the average age of customers grouped by gender."
execute_query(q3)

Here is the answer:

Question: calculates the average age of customers grouped by gender.
SQLQuery: SELECT `Gender`, AVG(`Age`) AS `AverageAge` FROM `sales_tb` GROUP BY `Gender` LIMIT 5;
###################################################
SELECT `Gender`, AVG(`Age`) AS `AverageAge` FROM `sales_tb` GROUP BY `Gender` LIMIT 5;
###################################################
###################################################
[('Male', Decimal('35.2143')), ('Female', Decimal('43.3333'))]


In [43]:
q4 = "identify the top spending customers based on their total amount spent."
execute_query(q4)

Here is the answer:

Question: identify the top spending customers based on their total amount spent.
SQLQuery: SELECT `CustomerID`, SUM(`TotalAmount`) AS `TotalSpent` FROM `sales_tb` GROUP BY `CustomerID` ORDER BY `TotalSpent` DESC LIMIT 5;
###################################################
SELECT `CustomerID`, SUM(`TotalAmount`) AS `TotalSpent` FROM `sales_tb` GROUP BY `CustomerID` ORDER BY `TotalSpent` DESC LIMIT 5;
###################################################
###################################################
[('CUST015', Decimal('2000.00')), ('CUST013', Decimal('1500.00')), ('CUST016', Decimal('1500.00')), ('CUST002', Decimal('1000.00')), ('CUST026', Decimal('1000.00'))]


In [44]:
q5 = "counts the number of transactions made each month."
execute_query(q5)

Here is the answer:

Question: counts the number of transactions made each month.
SQLQuery: SELECT MONTH(`Date`) AS `Month`, COUNT(`TransactionID`) AS `TransactionCount` FROM sales_tb GROUP BY MONTH(`Date`) ORDER BY `Month` LIMIT 5;
###################################################
SELECT MONTH(`Date`) AS `Month`, COUNT(`TransactionID`) AS `TransactionCount` FROM sales_tb GROUP BY MONTH(`Date`) ORDER BY `Month` LIMIT 5;
###################################################
###################################################
[(1, 4), (2, 4), (3, 1), (4, 5), (5, 2)]


In [45]:
q6 = "calculates the total sales amount and average price per unit for each product category."
execute_query(q6)

Here is the answer:

Question: calculates the total sales amount and average price per unit for each product category.
SQLQuery: SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSalesAmount`, AVG(`PriceperUnit`) AS `AveragePriceperUnit` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
SELECT `ProductCategory`, SUM(`TotalAmount`) AS `TotalSalesAmount`, AVG(`PriceperUnit`) AS `AveragePriceperUnit` FROM `sales_tb` GROUP BY `ProductCategory` LIMIT 5;
###################################################
###################################################
[('Beauty', Decimal('1455.00'), Decimal('153.750000')), ('Clothing', Decimal('5040.00'), Decimal('183.461538')), ('Electronics', Decimal('5310.00'), Decimal('238.750000'))]
